In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import time
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import statsmodels.formula.api as sm
import statsmodels.regression.linear_model as lm
import sys
sys.path.append('C:\\Users\\rcalde1\\Python_Columbia\\Final_project')
import exp
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split

# plotting modules
import seaborn as sns
import matplotlib.pyplot as plt



In [2]:
file = "features_45k.csv"
tds1 = pd.read_csv(file)
tds1 = tds1[['Artist', 'Track', 'followers','instrumentalness','popularity','acousticness', 'danceability', 'duration_ms', 'energy', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence']]
tds1.head()

,Artist,Track,followers,instrumentalness,popularity,acousticness,danceability,duration_ms,energy,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Lionel Richie,Tonight Will Be Alright,1627956,0.014100,38,0.05910,0.646,307413,0.840,8,0.0516,-8.213,1,0.0283,125.187,4,0.716
1,Course Of Nature,The Window,10469,0.000035,11,0.00119,0.496,191160,0.928,1,0.0988,-2.367,0,0.0469,144.932,4,0.561
2,Mariah Carey,Inseparable,5505368,0.000000,31,0.09090,0.586,213720,0.692,8,0.0567,-5.143,1,0.1770,158.041,4,0.457
3,Extra Virgin,One Night,880039,0.002050,0,0.00361,0.643,210667,0.896,0,0.9010,-6.662,1,0.0359,130.011,4,0.338
4,Joe and Will Ask?,Claymore,13,0.776000,2,0.00328,0.699,119053,0.974,9,0.2900,-5.576,0,0.1500,125.934,4,0.443


In [3]:

#to perform the train test split of the data, the train test split function is imported from sklearn
from sklearn.model_selection import train_test_split
#the given problem is a classificaton problem. Hence linear regression is used for ML algorithm
from sklearn.linear_model import LinearRegression


In [4]:
#Combined Dataset
dfcombine = tds1
#dfcombine = dfcombine.reset_index(inplace=True, drop=True)

#Target Dataset
dfy = pd.DataFrame(tds1["popularity"])

#all the independant variables/predictors are named as dfx
dfx = dfcombine.drop(columns=['Artist', 'Track','popularity'])


In [5]:
print(dfx.shape)
print(dfy.shape)

(45157, 14)
(45157, 1)


In [6]:
#Get rid of NaN, infinity or a value too large for dtype('float64').

np.where(dfx.values >= np.finfo(np.float64).max)
np.isnan(dfx.values.any())
dfx.replace([np.inf, -np.inf], np.nan, inplace=True)

np.where(dfy.values >= np.finfo(np.float64).max)
np.isnan(dfy.values.any())
dfy.replace([np.inf, -np.inf], np.nan, inplace=True)

dfx.reset_index(inplace=True, drop=True)
dfy.reset_index(inplace=True, drop=True)

print(dfx.shape)
print(dfy.shape)

(45157, 14)
(45157, 1)


In [7]:
#the percentage of the split is taken as 30%. SO the percentage of training is 70%
X_train, X_test, y_train, y_test = train_test_split(dfx, dfy, test_size=0.3, random_state=42)

In [8]:
# linear regression 
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [9]:
#the target is predicted for the test dataset
predictions = model.predict(X_test)
#the accuracy of the prediction is found to be.... 
model.score(X_test,y_test)

0.22164320209621935

# Pickle A: Linear Regression

In [10]:
# Import pickle Package
import pickle
# Save the Modle to file in the current working directory
Pkl_Filename = "Pickle_45_Model.pkl"  
with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)


In [11]:
# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)
Pickled_LR_Model


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [12]:
# Use the Reloaded Model to 
# Calculate the accuracy score and predict target values
# Calculate the Score 
score = Pickled_LR_Model.score(X_test, y_test)  
# Print the Score
print("Test score: {0:.2f} ".format(score))  
# Predict the Labels using the reloaded Model
Y_predict = Pickled_LR_Model.predict(X_test)  
Y_predict

Test score: 0.22 


array([[40.86042781],
       [36.20730164],
       [44.66136192],
       ...,
       [24.99936195],
       [53.1435068 ],
       [37.83834929]])

In [13]:
X_test.to_csv (r'C:\Users\Suhaib Kiani\Desktop\X_test.csv', index = False, header=True)

In [14]:
y_test.to_csv (r'C:\Users\Suhaib Kiani\Desktop\y_test.csv', index = False, header=True)

# Pickle B: Random_Forest

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [16]:
dataframe = tds1
print(dataframe.shape)
dataframe.describe()

(45157, 17)


,followers,instrumentalness,popularity,acousticness,danceability,duration_ms,energy,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
count,4.515700e+04,45157.000000,45157.000000,45157.000000,45157.000000,4.515700e+04,45157.000000,45157.000000,45157.000000,45157.000000,45157.000000,45157.000000,45157.000000,45157.000000,45157.000000
mean,1.890777e+06,0.124477,39.472994,0.278239,0.593009,2.310003e+05,0.631839,5.276015,0.190949,-8.223496,0.648094,0.090158,120.917113,3.930841,0.516775
std,5.217369e+06,0.274807,24.712035,0.307571,0.171259,8.312010e+04,0.232142,3.581349,0.163793,4.300239,0.477570,0.102903,29.012747,0.363697,0.255033
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,5.627000e+03,0.000020,0.000000,0.000000,-54.909000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.646100e+04,0.000000,17.000000,0.019000,0.482000,1.870670e+05,0.475000,2.000000,0.093100,-10.197000,0.000000,0.035400,98.257000,4.000000,0.313000
50%,1.751950e+05,0.000060,43.000000,0.141000,0.606000,2.191200e+05,0.662000,5.000000,0.124000,-7.256000,1.000000,0.049200,120.014000,4.000000,0.515000
75%,1.075210e+06,0.021200,60.000000,0.490000,0.720000,2.594130e+05,0.822000,8.000000,0.240000,-5.261000,1.000000,0.093700,138.841000,4.000000,0.725000
max,6.237875e+07,0.999000,98.000000,0.996000,0.988000,2.752254e+06,1.000000,11.000000,0.998000,4.405000,1.000000,0.963000,245.941000,5.000000,0.998000


In [17]:
dataframe.loc[dataframe['popularity'] < 60, 'popularity'] = 0 
dataframe.loc[dataframe['popularity'] >= 60, 'popularity'] = 1
dataframe['popularity'].value_counts()

0    33443
1    11714
Name: popularity, dtype: int64

In [18]:
features_x = ['followers','instrumentalness','acousticness', 'danceability', 'duration_ms', 'energy', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence']

In [19]:
#Next we define 80% of the dataframe for training and 20% of the dataframe for testing.
training = dataframe.sample(frac = 0.8,random_state = 420)
X_train = training[features_x]
y_train = training['popularity']
X_test = dataframe.drop(training.index)[features_x]

In [20]:
#We add a validation dataset using train_test_split.
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

In [21]:
RFC_Model = RandomForestClassifier()
RFC_Model.fit(X_train, y_train)
RFC_Predict = RFC_Model.predict(X_valid)
RFC_Accuracy = accuracy_score(y_valid, RFC_Predict)
print("Accuracy: " + str(RFC_Accuracy))

RFC_AUC = roc_auc_score(y_valid, RFC_Predict) 
print("AUC: " + str(RFC_AUC))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Accuracy: 0.9100470523110988
AUC: 0.8692357254382671


In [22]:
RFC_Model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [23]:
# Import pickle Package
import pickle
# Save the Modle to file in the current working directory
Pkl_Filename2 = "Pickle_45_Model2.pkl"  
with open(Pkl_Filename2, 'wb') as file2:  
    pickle.dump(RFC_Model, file2)


In [24]:
# Load the Model back from file
with open(Pkl_Filename2, 'rb') as file2:  
    Pickled_LR_Model2 = pickle.load(file2)
Pickled_LR_Model2


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [25]:
RFC_Model2 = Pickled_LR_Model2
RFC_Model2.fit(X_train, y_train)
RFC_Predict2 = RFC_Model2.predict(X_valid)
RFC_Accuracy2 = accuracy_score(y_valid, RFC_Predict2)
print("Accuracy: " + str(RFC_Accuracy2))
RFC_AUC2 = roc_auc_score(y_valid, RFC_Predict2) 
print("AUC: " + str(RFC_AUC2))

Accuracy: 0.9153058400221422
AUC: 0.8722952941874025


In [26]:
X_valid.to_csv (r'C:\Users\Suhaib Kiani\Desktop\X_valid.csv', index = False, header=True)
y_valid.to_csv (r'C:\Users\Suhaib Kiani\Desktop\y_valid.csv', index = False, header=True)